# L4: Tools for a Customer Outreach Campaign

## SETUP

In [1]:
# Warning control
import warnings
warnings.filterwarnings("ignore")

In [2]:
from crewai import Agent, Task, Crew

In [7]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

## AGENTS

In [11]:
agent_sales_rep = Agent(
    role="""Sales Representative""",
    goal="""Identify high-value leads that match our ideal customer profile""",
    backstory=(
        """
As a part of the dynamic sales team at CrewAI, your mission is to scour the digital landscape for potential leads. Armed with cutting-edge tools and a strategic mindset, you analyze data, trends, and interactions to unearth opportunities that others migh overlook. Your work is crucial in paving the way for meaningful engagements and driving the company's growth.
"""
    ),
    allow_delegation=False,
    verbose=True,
)

# agent_sales_rep

In [10]:
agent_lead_sales_rep = Agent(
    role="""Lead Sales Representative""",
    goal="""Nurture leads with personalized, compelling communications""",
    backstory=(
        """Within the vibrant ecosystem of CrewAI's sales department, you stand out as the bridge between potential clients and the solutions they need. By creating engaging, personalized messages, you not only inform leads about our offerings but also make them feel seen and heard. Your role is pivotal in converting interest into action, guiding leads through the journey from curiosity to commitment."""
    ),
    allow_delegation=False,
    verbose=True,
)

## TOOLS

In [18]:
# crewai tools
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

tool_directory_read = DirectoryReadTool("../files/instructions")
tool_file_read = FileReadTool()
tool_search = SerperDevTool()

In [17]:
# custom tools
from crewai_tools import BaseTool


class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = (
        "Analyzes the sentiment of text to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # custom code
        return "positive"


tool_sentiment_analysis = SentimentAnalysisTool()

## TASKS

In [22]:
task_lead_profiling = Task(
    description=(
        """Conduct an in-depth analysis of {lead_name}, a company in the {industry} sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision makers, recent business development, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively. \n Don't make assumptions and only use information you absolutely sure about."""
    ),
    expected_output=(
        """A comprehensive report on {lead_name}, including company background, key personnel, recent milestones and identified needs. Highlight potential areas where our solutions can provide value and suggest personalized engagement strategies."""
    ),
    tools=[tool_directory_read, tool_file_read, tool_search],
    agent=agent_sales_rep,
)

In [27]:
task_personalized_outreach = Task(
    description=(
        """Using the insights gathered from the lead profiling report on {lead_name}, craft a personalized outreach campaign aimed at {key_decision_maker}, the {position} of {lead_name}. The campaign should address their recent {milestone} and how our solutions can support their goals. Your communication must resonate with {lead_name}'s company culture and values, demonstrating a deep understanding of their business and needs. \n Don't make assumptions and only use information you absolutely sure about."""
    ),
    expected_output=(
        """A series of personalized email drafts tailored to {lead_name}, specifically targeting {key_decision_maker}. Each drafat should include a compelling narrative that connects our solutions with their recent achievements and future goals. Ensure the tone is engaging, professional, and aligned with {lead_name}'s corporate identity"""
    ),
    tools=[tool_sentiment_analysis, tool_search],
    agent=agent_lead_sales_rep,
)

## CREW

In [32]:
# define the crew
crew = Crew(
    agents=[agent_sales_rep, agent_lead_sales_rep],
    tasks=[task_lead_profiling, task_personalized_outreach],
    verbose=0,
    memory=True,
)

2024-05-26 09:09:48,013 - 8610696192 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [33]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Anrew Ng",
    "position": "CEO",
    "milestone": "Product launch",
}

result = crew.kickoff(inputs=inputs)



> Entering new CrewAgentExecutor chain...
I should first search the internet for general information about DeepLearningAI and its key decision makers to start building a basic profile.

Action: 
Search the internet

Action Input: 
{"search_query": "DeepLearningAI company profile"} 
 


Search results: Title: Deeplearning.ai - Crunchbase Company Profile & Funding
Link: https://www.crunchbase.com/organization/deeplearning-ai
Snippet: Deeplearning.ai is company that explores the frontier of AI. The company is founded by Andrew Ng, the former Chief Scientist of Baidu and the current Chairman ...
---
Title: DeepLearning.AI - LinkedIn
Link: https://www.linkedin.com/company/deeplearningai
Snippet: DeepLearning.AI is making a world-class AI education accessible to people around the globe. DeepLearning.AI was founded by Andrew Ng, a global leader in AI.
---
Title: DeepLearning.AI: Start or Advance Your Career in AI
Link: https://www.deeplearning.ai/
Snippet: Learn the skills to start or advan

In [34]:
from IPython.display import Markdown

Markdown(result)

"Dear Dr. Ng, 

I hope this message finds you well. I am writing to you as the Lead Sales Representative at CrewAI. First of all, I want to congratulate you on the successful launch of 'Generative AI for Everyone.' It's truly an impressive feat and speaks volumes about DeepLearningAI's commitment to making world-class AI education accessible to all. 

In line with your mission, CrewAI offers solutions that could significantly enhance the online learning experience and facilitate AI course creation and delivery. We believe that our offerings are uniquely positioned to support DeepLearningAI's goal of empowering the global workforce through AI education. 

I would appreciate the opportunity to discuss how our solutions could integrate with your current offerings and help you achieve your future goals. 

Looking forward to hearing from you soon. 

Best regards, 
[Your Name]"